### Evaluation notebook

In [ ]:
"""
Run 'train_model.py' or 'train_model.ipynb' first...
"""

import os
import sys
from importlib.resources import files

from optimetal.evaluation import Evaluator
from optimetal.data.loader import load_torch_data, create_dataloader

"""
-----------------------------------------------------------------------------------------------------------------------
START OF USER INPUT:
"""

# path to a 'best_model.pt' file created by 'optimetal.training.Trainer'
model_name = "optimetal3b_seed42"
best_model_path = files("optimetal.files").joinpath(f"{model_name:s}.pt")

# path to the dataset we want to evaluate
eval_path = "../graph/test.pt" # 'val.pt' or 'test.pt'

# batch size (speeds up the process at bit)
batch_size = 32 # adjust this according to your GPU memory

# device index (0 for GPU 0, 1 for GPU 1, etc. or -1 for CPU)
device_index = 0

# plot directory
eval_result_dir = f"./best_model_eval/{model_name:s}"

"""
END OF USER INPUT:
-----------------------------------------------------------------------------------------------------------------------
"""

# sanity checks
if not os.path.exists(eval_path):
    sys.exit("The path 'eval_path' does not exist (evaluation data not found)")
if batch_size < 2:
    raise ValueError("The 'batch_size' must be larger than 1")

# output directory setup
os.makedirs(eval_result_dir, exist_ok=True)

In [ ]:
# load the data on which we want to evaluate the model
eval_data = load_torch_data(eval_path)
dataloader = create_dataloader(
    eval_data, 
    num_data=-1, # use the whole dataset 
    batch_size=batch_size,
    shuffle=False, # do not shuffle
)
print(f"Loaded evaluation data from '{eval_path:s}'", flush=True)
    
# setup the evaluator
evaluator = Evaluator(
    best_model_path=best_model_path, 
    dataloader=dataloader,
    device_index=device_index,
    dataset_name=eval_path.split("/")[-1].split(".")[0]
)

In [ ]:
# evaluate the model
evaluator.evaluate()

In [ ]:
# print and store metrics
# (all results/plots/files will found in the 'eval_result_dir' directory)
evaluator.print_metrics()
evaluator.store_metrics(metric_path=os.path.join(eval_result_dir, "metrics.json"))

In [ ]:
# make a nice quantile plots
fig, gs = evaluator.quantile_plot(fig_path=os.path.join(eval_result_dir, "quantiles.pdf"), rng_seed=42)
fig, gs = evaluator.quantile_plot(interband_only=True, fig_path=os.path.join(eval_result_dir, "quantiles_interband.pdf"), rng_seed=42)

In [ ]:
#  plots dielectric function of some random materials
evaluator.store_metrics(metric_path=os.path.join(eval_result_dir, "metrics.json"))
fig, ax = evaluator.plot_rand_mats(
    num_mats=16, 
    fig_path=os.path.join(eval_result_dir, "rand_mats.pdf"),
    rng_seed=42,
)
fig, ax = evaluator.plot_rand_mats(
    num_mats=16, 
    interband_only=True,
    fig_path=os.path.join(eval_result_dir, "rand_mats_interband.pdf"),
    rng_seed=42,
)

In [ ]:
print(f"Available metrics for histograms and violin plots")
for key in evaluator.label_dict:
    print(f'"{key:s}"')

In [ ]:
# analyze the distribution of a metrics (just an example...)
metric = "loss"
fig, ax = evaluator.metric_histogram(metric)
fig, ax = evaluator.metric_violinplot(metric)

In [ ]:
# print the metric table in a latex format, ready to copy
latex_table_str = evaluator.print_metric_latex_table()
with open(os.path.join(eval_result_dir, "latex_table.txt"), "w") as f:
    f.write(latex_table_str)